In [1]:
from data.neuro_mnist_loader import load_n_mnist_batches

In [2]:
batches_dataset = load_n_mnist_batches()

In [3]:
train_loader = batches_dataset.train_loader
print(train_loader)

In [13]:
for image, label in train_loader:
    print(image.shape, label.shape)
    break

torch.Size([128, 150, 2312]) torch.Size([128])


In [16]:
import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import itertools
from utils.globals import device, BATCH_SIZE


In [17]:
spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.9

In [83]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=12, kernel_size=3, padding=1)
        self.lif1 = snn.Leaky(beta=beta, spike_grad=spike_grad)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, padding=1)
        self.lif2 = snn.Leaky(beta=beta, spike_grad=spike_grad)
        self.fc1 = nn.Linear(in_features=24*8, out_features=10)
        self.lif3 = snn.Leaky(beta=beta, spike_grad=spike_grad)

    def forward(self, x):

        # Initialize hidden states and outputs at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        
        cur1 = F.max_pool2d(self.conv1(x), 2)
        spk1, mem1 = self.lif1(cur1, mem1)

        print(spk1.shape, mem1.shape)
        cur2 = F.max_pool2d(self.conv2(spk1), 2)
        spk2, mem2 = self.lif2(cur2, mem2)
        print(spk2.shape, mem2.shape)

        cur3 = self.fc1(spk2.view(BATCH_SIZE, -1))
        spk3, mem3 = self.lif3(cur3, mem3)
        return spk3, mem3

In [81]:
net = Net().to(device)

In [52]:
net

Net(
  (conv1): Conv2d(2, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (lif1): Leaky()
  (conv2): Conv2d(12, 64, kernel_size=(5, 5), stride=(1, 1))
  (lif2): Leaky()
  (fc1): Linear(in_features=1024, out_features=10, bias=True)
  (lif3): Leaky()
)

In [82]:
data, targets = next(iter(train_loader))


data = data.to(device)
targets = targets.to(device)

data = data.view(data.shape[0], data.shape[1], -1)
print(data.shape)
data = data[0, 0, :].view(2, 34, 34)
print(data.shape)

net(data)

torch.Size([128, 150, 2312])
torch.Size([2, 34, 34])
torch.Size([12, 17, 17]) torch.Size([12, 17, 17])
torch.Size([24, 8, 8]) torch.Size([24, 8, 8])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x24 and 192x10)